# Advent of Code 2025 - Day 7

In [1]:
import numpy as np

# Test data
test = """.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
..............."""

## Part 1


In [2]:
def count_splits_recursive(grid, row, beam_positions):
    """
    Recursively count splits in the grid.
    row: current row index (where beams are currently located)
    beam_positions: set of column indices where beams are located in this row
    """
    if row >= len(grid):
        return 0

    splits = 0
    next_beam_positions = set()

    # Process each beam position
    for col in beam_positions:
        # Check if there's a splitter at this position in the current row
        if 0 <= col < len(grid[row]) and grid[row][col] == "^":
            splits += 1
            # Beam stops here, two new beams start at next row (left and right)
            if row + 1 < len(grid):
                if col - 1 >= 0:
                    next_beam_positions.add(col - 1)  # left beam
                if col + 1 < len(grid[row + 1]):
                    next_beam_positions.add(col + 1)  # right beam
        elif 0 <= col < len(grid[row]) and grid[row][col] in [".", "S"]:
            # Beam continues straight down (no splitter, just empty space or S)
            if row + 1 < len(grid) and 0 <= col < len(grid[row + 1]):
                next_beam_positions.add(col)

    # Continue recursively with next row and new beam positions
    if next_beam_positions:
        return splits + count_splits_recursive(grid, row + 1, next_beam_positions)
    else:
        return splits

In [3]:
# Test example
test_rows = test.strip().split("\n")
max_len = max(len(row) for row in test_rows)
test_grid = [np.array(list(row.ljust(max_len))) for row in test_rows]

start_col = np.where(test_grid[0] == "S")[0][0]
beam_positions = {start_col}
splits = count_splits_recursive(test_grid, 0, beam_positions)
print(f"Test result: {splits}")

Test result: 21


In [4]:
with open("input.txt", "r") as file:
    data = file.read().strip()

rows = data.strip().split("\n")
max_len = max(len(row) for row in rows)
grid = [np.array(list(row.ljust(max_len))) for row in rows]

start_col = np.where(grid[0] == "S")[0][0]
beam_positions = {start_col}
splits = count_splits_recursive(grid, 0, beam_positions)
print(f"Part 1 answer: {splits}")


Part 1 answer: 1667


## Part 2


In [5]:
def count_timelines(grid):
    """
    Count the total number of distinct timelines using dynamic programming.
    Each cell (row, col) stores the number of distinct timelines that reach it.
    """
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    # Initialize timeline counts: timelines[row][col] = number of timelines reaching this cell
    timelines = [[0] * cols for _ in range(rows)]

    # Find starting position
    start_col = np.where(grid[0] == "S")[0][0]
    timelines[0][start_col] = 1  # Start with 1 timeline

    # Process each row from top to bottom
    for row in range(rows):
        for col in range(cols):
            if timelines[row][col] == 0:
                continue  # No timelines reach this cell

            num_timelines = timelines[row][col]
            cell = grid[row][col]

            if cell == "^":
                # Splitter: timelines split into left and right
                # Each timeline goes both left and right (quantum behavior)
                if row + 1 < rows:
                    if col - 1 >= 0:
                        timelines[row + 1][col - 1] += num_timelines  # Left path
                    if col + 1 < cols:
                        timelines[row + 1][col + 1] += num_timelines  # Right path
            elif cell in [".", "S"]:
                # Empty space or start: timelines continue straight down
                if row + 1 < rows:
                    timelines[row + 1][col] += num_timelines

    # Sum all timelines that reach the bottom row (or exit the grid)
    total_timelines = sum(timelines[rows - 1])
    return total_timelines


In [6]:
# Test example
test_rows = test.strip().split("\n")
max_len = max(len(row) for row in test_rows)
test_grid = [np.array(list(row.ljust(max_len))) for row in test_rows]

timelines = count_timelines(test_grid)
print(f"Test result: {timelines}")


Test result: 40


In [7]:
with open("input.txt", "r") as file:
    data = file.read().strip()

rows = data.strip().split("\n")
max_len = max(len(row) for row in rows)
grid = [np.array(list(row.ljust(max_len))) for row in rows]

timelines = count_timelines(grid)
print(f"Part 2 answer: {timelines}")


Part 2 answer: 62943905501815
